In [1]:
from torch.utils.data import DataLoader
from data.hfdataset import HFImageDataset
from data.transforms.geoclip import geoclip_train_transform
from model.embedding_heads.clip_image import CLIPImageEncoder
from model.embedding_heads.clip_text import CLIPTextEncoder
from model.embedding_heads.dino_image import DINOImageEncoder
from model.embedding_heads.rff_location import RFFLocationEncoder
from model.GeoClip import GeoCLIP

In [2]:
dataset = HFImageDataset('GeoGuessing/GeoTaggedImages', 'train[:-16]', transform=geoclip_train_transform())

In [3]:
dataset.set_columns('image', 'lat', 'lon')

In [4]:
dataloader = DataLoader(dataset, batch_size=256, shuffle=True)

In [5]:
location_encoder = RFFLocationEncoder()

In [6]:
image_encoder = CLIPImageEncoder()

In [7]:
location_encoder.load_weights('/Users/kshitij/Documents/University/Year4/MLP/RealProject/weights/pretrained_geoclip/location_encoder_weights.pth')
image_encoder.load_weights('/Users/kshitij/Documents/University/Year4/MLP/RealProject/weights/pretrained_geoclip/image_encoder_mlp_weights.pth')

/Users/kshitij/Documents/University/Year4/MLP/RealProject/model/embedding_heads/rff_location.py:219: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.load_state_dict(torch

In [8]:
model = GeoCLIP(image_encoder, location_encoder, '/Users/kshitij/Documents/University/Year4/MLP/RealProject/weights/pretrained_geoclip/coordinates_100K.csv')

In [9]:
model.load_weights('/Users/kshitij/Documents/University/Year4/MLP/RealProject/weights/pretrained_geoclip/logit_scale_weights.pth')

/Users/kshitij/Documents/University/Year4/MLP/RealProject/model/GeoClip.py:38: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.logit_scale = nn.Parameter(torch.load(path)

In [10]:
# Image Upload & Display
from PIL import Image
from io import BytesIO
# from google.colab import files
import matplotlib.pyplot as plt

# Heatmap
import folium
from folium.plugins import HeatMap

model.to('mps')
# Make predictions
top_pred_gps, top_pred_prob = model.predict("download.png", top_k=5)
print(top_pred_gps.shape)
# Display the top 5 GPS predictions
print("Top 5 GPS Predictions 📍")
print("========================")
for i in range(5):
    lat, lon = top_pred_gps[i]
    print(f"Prediction {i+1}: ({lat:.6f}, {lon:.6f}) - Probability: {top_pred_prob[i]:.6f}")

torch.Size([5, 2])
Top 5 GPS Predictions 📍
Prediction 1: (51.327446, -116.183510) - Probability: 0.087768
Prediction 2: (51.326401, -116.182632) - Probability: 0.086992
Prediction 3: (51.328197, -116.180931) - Probability: 0.084368
Prediction 4: (51.327808, -116.180336) - Probability: 0.083671
Prediction 5: (51.322353, -116.185570) - Probability: 0.080773


In [ ]:
from geoclip import GeoCLIP as RealGeoCLIP

model2 = RealGeoCLIP()
model2.to('cpu')

